# Sensor Simulator

In the previous notebook
[`2.1-bayesian-optimization-blooper.ipynb`](2.1-bayesian-optimization-blooper.ipynb), we ran
into some unexpected results. Bayesian optimization was on par with grid search, and
random search was the best? See below.

![blooper](bayesian-optimization-blooper.png)

> SPOILER: In this and the next notebook, we'll see how this was an issue with how the
> pre-processing for plotting was being carried out. Also, extended use of the hardware
> may have raised the temperature, leading to larger variation in the measured readings

To help troubleshoot the source of the unexpected
behavior, we'll move the schedule up and use a simulator. The large stochasticity for
identical inputs shown at the end of the last notebook is a cause for concern.

|  | mean | std |
|---|---|---|
| ch415_violet | 17928.500000 | 25288.711738 |
| ch445_indigo | 8746.200000 | 9379.586248 |
| ch480_blue | 19396.100000 | 17832.023101 |
| ch515_cyan | 4870.900000 | 12126.813252 |
| ch560_green | 11908.300000 | 18944.797128 |
| ch615_yellow | 18406.400000 | 19626.981147 |
| ch670_orange | 30584.100000 | 20868.707815 |
| ch720_red | 5776.100000 | 9066.354228 |
| ch_clear | 1294.100000 | 2017.116669 |
| ch_nir | 0.000000 | 0.000000 |
| mae | 13798.390000 | 2959.743242 |
| rmse | 23851.888103 | 3422.810056 |

Is the stochasticity due to excessive noise from too short of integration time or is it
a problem with the hardware such as a sensor malfunction? Based on inspection from my
own eyes, fixed inputs appeared to produce very similar colors. Last, this could be due
to (intentional) naive design decision of starting out with mean absolute error for the
objective function rather than something more tuned to this problem, such as Wasserstein
distance between the two discrete spectra. In reality, it's probably some mix and
interplay of the previous issues:

- **(epistemic) uncertainty**
  - i.e. just a part of the system, solved by a longer integration time (more data)
- **sensor malfunction/degradation**
  - due to blasting it repeatedly with a bright DotStar
  - and/or hard device resets due to system crashes
  - and/or or (just remembered!) a series
  of recent unexpected power outages for my housing community
- **Poorly defined objective function**
  - It was defined in a way that it doesn't handle stochastic bias when
  a signal is measured in an adjacent channel, solved by using a robust distance metric
  for discrete distributions (e.g. Wasserstein)
- **Something else?**

My wife kindly reminded me of past lessons learned during experimentation, and
encouraged me to try out the simulation. So, let's do that here! We'll take a look at
our domain knowledge / assumptions for the simulation, briefly describe the approach,
and then dig into a snapshot of the `SensorSimulator` class. Finally, we'll demonstrate
the usage and run our grid vs. random vs. Bayesian search algorithm comparison with our
simulation.


## Domain Knowledge

> TLDR; extract the data from the DotStar RGB spectrum listed in the [manufacturer's datasheet](https://cdn-shop.adafruit.com/product-files/2343/SK9822_SHIJI.pdf)

A basic physics course teaches the wave-particle duality of light. Light can be thought
of as a wave (signal with frequency/wavelength) or as a particle (photon). In the case
of designing a (very basic) simulation for an RGB LED spectrum being read by a
discrete-channel sensor, we focus mainly on the wave portion of wave-particle duality of
light. Most light-emitting-diodes (LEDs) have fairly narrow distributions of wavelengths
that they emit like what's shown in the following image from the [DotStar manufacturer's datasheet](https://cdn-shop.adafruit.com/product-files/2343/SK9822_SHIJI.pdf).

<img src="../reports/figures/dotstar/rgb-relative-emission-vs-wavelength.png" width=350>

Controlling the brightness and RGB values of our LED controls the relative contribution
of each of the three distributions portrayed above. Meanwhile, our spectrophotometer (a device that measures the intensity of light at various
wavelengths) measures the light intensity at 8 wavelengths (it also has "clear" and
"near infrared" channels that we'll ignore):

| wavelength | intensity |
|---|---|
415 nm | Violet
445 nm | Indigo or Blue
480 nm | Blue or Blue-Green
515 nm | Blue-Green or Green
555 nm | Green or Yellow-Green 
590 nm | Yellow-Green or Yellow
630 nm | Orange or Orange-Red or Red
680 nm | Red

We can digitize the data from the manufacturer datasheet using
[WebPlotDigitizer](https://automeris.io/WebPlotDigitizer/) and then use that for
calculating/mixing an arbitrary spectrum based on brightness and RGB settings. We'll make the
reasonable approximation/assumption that the [superposition principle](https://en.wikipedia.org/wiki/Superposition_principle) applies*. In other
words, we assume that the individual spectra from the red, green, and blue LEDs can be
added together. Below is a screenshot of the datapoints extracted via the
WebPlotDigitizer interface for each of the three curves. Corresponding `.csv` files are
also saved: [`red.csv`](../src/self_driving_lab_demo/data/red.csv), [`green.csv`](../src/self_driving_lab_demo/data/green.csv), and [`blue.csv`](../src/self_driving_lab_demo/data/blue.csv).

<img src="../src/self_driving_lab_demo/data/wpd-rgb-spectrum-points-overlay.png"
width=350>

See [RGB LEDs vs. having 10+ monochromatic light
sources](https://github.com/sparks-baird/self-driving-lab-demo/issues/6) for more
details on the hardware design considerations for the LEDs and sensor.

<p><sup>
*While it would be interesting to look at optimization setups that
involve light cancellation, the equipment required is likely not within the budget
and time constraints of the self-driving-lab-demo project (less than 100 USD and less
than an hour of setup time).
</sup></p>


## Simulation Details

As mentioned, we'll mix the three spectra extracted from the manufacturer datasheet
according to the simulation inputs (brightness
and RGB values). After loading the CSV data, we'll clip any data below 0.0 and average
the intensities that are not one-to-one (i.e. multiple intensities for the same
wavelength) due to the data extraction process. Next, we'll use `scipy`'s `interp1d`
function for each of the three color spectra to create linear interpolation functions
which are set to zero outside of the range of the original data (extrapolation).
Finally, we'll calculate a weighted sum of each of the interpolators as a function of
brightness and RGB sampled at each of the wavelengths mentioned above.

A more sophisticated setup might involve sampling a distribution of wavelengths for each
channel; however, we'll keep it simple for now.

## Sensor Simulator Python class
Below is a snapshot of the `SensorSimulator` class, broken into chunks.

First, we need our imports and define the wavelengths we'll be sampling at as a constant.

```python
from importlib.resources import open_text
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from self_driving_lab_demo import data as data_module

CHANNEL_WAVELENGTHS = [
    415,
    445,
    480,
    515,
    560,
    615,
    670,
    720,
]
```

The class doesn't take any keyword arguments (again, keeping it simple), and when the
class is instantiated, it creates an interpolator.

```python
class SensorSimulator(object):
    def __init__(self):
        self.red_interp = self.create_interpolator("red.csv")
        self.green_interp = self.create_interpolator("green.csv")
        self.blue_interp = self.create_interpolator("blue.csv")

```

We make it easy to get the channel wavelengths (making it a constant outside the class
makes it easily accessible for other modules).

```python
    @property
    def channel_wavelengths(self):
        return CHANNEL_WAVELENGTHS
```

The data is read using best practices (`open_text` using Python modules), negative
values are clipped, and y-values with repeat x-values are averaged. Finally, the
`interp1d` uses a linear interpolation (`cubic` gave some outlandish values during
testing) and zero-ing extrapolation (zero anywhere outside the range of the original
dataset). This function operates on a single color curve.

```python
    def create_interpolator(self, fname):
        df = pd.read_csv(
            open_text(data_module, fname),
            header=None,
            names=["wavelength", "relative_intensity"],
        )

        df["relative_intensity"].clip(lower=0.0, inplace=True)

        # average y-values for repeat x-values
        # see also https://stackoverflow.com/a/51258988/13697228
        df = df.groupby("wavelength", as_index=False).mean()

        return interp1d(
            df["wavelength"],
            df["relative_intensity"],
            kind="linear",
            bounds_error=False,
            fill_value=0.0,
        )

```

To perform the weighted average (mixing) of spectra, we divide the RGB values by 255
(this is arbitrary) and multiply by the brightness. This is then multipled by the
interpolated value at each of the wavelengths. Finally, we sum the contribution of each
of the wavelengths.

```python
    def _simulate_sensor_data(self, wavelengths, brightness, R, G, B):
        rI, gI, bI = brightness * np.array([R, G, B]) / 255
        channel_data = np.sum(
            [
                self.red_interp(wavelengths) * rI,
                self.green_interp(wavelengths) * gI,
                self.blue_interp(wavelengths) * bI,
            ],
            axis=0,
        )
        return tuple(channel_data)
```

Then, we define a class method that fixes the wavelengths to the constant mentioned before.

```python
    def simulate_sensor_data(self, brightness, R, G, B):
        return self._simulate_sensor_data(self.channel_wavelengths, brightness, R, G, B)
```

Last, let's take a look at how this is integrated into the `SelfDrivingLabDemo` class
(another reasonable design choice might be to create separate class using class
inheritance).

It's important that we only use the RPi-specific modules when running on RPi, otherwise
an error such as `NotImplementedError` will be thrown.

```python
try:
    import board
    from adafruit_as7341 import AS7341
    from blinkt import clear, set_brightness, set_pixel, show
except NotImplementedError as e:
    print(e)
    _logger.warning(
        "Safe to ignore if this is CI or not on a Raspberry Pi. However, only the simulator will be available."  # noqa: E501
    )
```

We add a `simulation` boolean to the `__init__` constructor.

```python
class SelfDrivingLabDemo(object):
    def __init__(
        self,
        ...
        simulation=False,
    ):
```

We store the kwarg as a class attribute, instantiate a `SensorSimulator` object, and
store the object as a class attribute.
```python
        ...
        self.simulation = simulation
        self.simulator = SensorSimulator()
```

It's important that we don't use the `board` and `AS7341` modules if we're not running
on a Raspberry Pi, so we assign `i2c` and `sensor` as `None` if those modules aren't
available (see `try` statement above).

```python
        # uses board.SCL and board.SDA
        self.i2c = None if "board" not in sys.modules else board.I2C()
        self.sensor = None if "AS7341" not in sys.modules else AS7341(self.i2c)
        ...
```

We make a slight modification to `observe_sensor_data` that calls the
`simulate_sensor_data` function if `self.simulation` is `True`.

```python
    def observe_sensor_data(self, brightness, R, G, B):
        if self.simulation:
            return self.simulate_sensor_data(brightness, R, G, B)
        ...

    def simulate_sensor_data(self, brightness, R, G, B):
        return self.simulator.simulate_sensor_data(brightness, R, G, B)
    ...
```

Note that in later versions, the details may differ somewhat from the above example, and
more features might be added.

In [22]:
%load_ext autoreload
%autoreload 2
from self_driving_lab_demo.core import SensorSimulator
sim = SensorSimulator()
discrete_spectrum = sim.simulate_sensor_data(0.5, 12, 24, 48)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


If we supply wavelengths directly, we can visualize the raw simulated data (i.e. the
full spectrum).

In [19]:
import numpy as np
import pandas as pd
import plotly.express as px
wavelengths = np.arange(400, 800, 0.1)
data = sim._simulate_sensor_data(wavelengths, 0.5, 12, 24, 48)
df = pd.DataFrame(dict(wavelength=wavelengths, intensity=data))
fig = px.scatter(data_frame=df, x="wavelength", y="intensity")
fig.update_traces(marker=dict(color="black"))
fig

Example output:

![simulator-data](simulator-data.png)

Next, we can show where the discrete wavelengths are being sampled.

In [26]:
import plotly.graph_objects as go
fig2 = go.Figure(fig)
[
    fig2.add_vline(x=ch, line_width=3, line_dash="dash", line_color=c)
    for ch, c in zip(sim.channel_wavelengths, sim.channel_hex_colors)
]
fig2

Example output:

![simulator-discrete-sampling](simulator-discrete-sampling.png)

And finally, we show the discrete spectrum.

In [29]:
df = pd.DataFrame(dict(wavelength=sim.channel_wavelengths, intensity=discrete_spectrum))
px.bar(data_frame=df, x="wavelength", y="intensity")

Example output:

![simulator-discrete-bar](simulator-discrete-bar.png)

In [ ]:
ss = SensorSimulator()
wavelengths = np.arange(400, 800, 0.1)
data = ss._simulate_sensor_data(
    wavelengths,
    brightness=0.5,
    R=12,
    G=24,
    B=48,
)

In [2]:
from self_driving_lab_demo.core import SelfDrivingLabDemo
sdl = SelfDrivingLabDemo(autoload=True, simulation=True)
sdl.evaluate(0.5, 50, 150, 250)

{'ch415_violet': 0.00018751611385529196,
 'ch445_indigo': 0.02978594739011109,
 'ch480_blue': 0.02911825030328232,
 'ch515_cyan': 0.025286431748673996,
 'ch560_green': 0.0,
 'ch615_yellow': 0.01497882456394869,
 'ch670_orange': 0.0006279784042173006,
 'ch720_red': 0.0,
 'mae': 0.010227082296880314,
 'rmse': 0.01456700897508149}

## Algorithm Comparison

We'll skip over some of the content that was covered in more detail in
[`2.1-bayesian-optimization-blooper.ipynb`](2.1-bayesian-optimization-blooper.ipynb).

### Setup

We'll instantiate our `num_repeats` `SelfDrivingLabDemo` instances, but this time with the
`simulation` flag set to `True`.

In [3]:
import numpy as np
import pandas as pd

num_iter = 81
num_repeats = 20
SEEDS = range(10, 10 + num_repeats)

sdls = [
    SelfDrivingLabDemo(autoload=True, simulation=True, target_seed=seed)
    for seed in SEEDS
]


The target inputs used to create the target spectrum are shown below.

In [4]:
target_inputs = [sdl.get_target_inputs() for sdl in sdls]
pd.DataFrame(target_inputs, columns=["brightness", "R", "G", "B"])

,brightness,R,G,B
0,0.334601,53,211,38
1,0.045000,127,153,7
2,0.087789,241,48,46
3,0.302679,218,207,67
4,0.290844,92,179,219
5,0.242460,208,88,11
6,0.198421,110,24,89
7,0.295776,41,142,94
8,0.139757,183,72,21
9,0.147133,236,70,15


In [5]:
target_data = [sdl.target_data for sdl in sdls]
pd.DataFrame(target_data, columns=sdls[0].channel_names)

,ch415_violet,ch445_indigo,ch480_blue,ch515_cyan,ch560_green,ch615_yellow,ch670_orange,ch720_red
0,1.907387e-05,0.003030,0.009847,0.023803,0.0,0.010625,0.000445,0.0
1,4.725361e-07,0.000075,0.000805,0.002321,0.0,0.003424,0.000144,0.0
2,6.057931e-06,0.000962,0.001137,0.001421,0.0,0.012676,0.000531,0.0
3,3.042187e-05,0.004832,0.010244,0.021124,0.0,0.039535,0.001657,0.0
4,9.555052e-05,0.015178,0.016351,0.017553,0.0,0.016032,0.000672,0.0
5,4.000937e-06,0.000636,0.002779,0.007194,0.0,0.030216,0.001267,0.0
6,2.649147e-05,0.004208,0.003476,0.001606,0.0,0.013077,0.000548,0.0
7,4.170803e-05,0.006625,0.009245,0.014160,0.0,0.007266,0.000305,0.0
8,4.402724e-06,0.000699,0.001591,0.003393,0.0,0.015324,0.000642,0.0
9,3.310768e-06,0.000526,0.001495,0.003472,0.0,0.020805,0.000872,0.0


### Grid and Random Search

We'll use some helper functions for grid and random search based on the implementation
in [`2.1-bayesian-optimization-blooper.ipynb`](2.1-bayesian-optimization-blooper.ipynb). Notice
how much faster our simple simulation runs compared with the experiment! (seconds vs. ~10 minutes)

In [6]:
from self_driving_lab_demo.utils.search import grid_search, random_search

In [7]:
%%time
grid_results = [grid_search(sdl, num_iter) for sdl in sdls]
random_results = [random_search(sdl, num_iter) for sdl in sdls]

CPU times: user 6.58 s, sys: 103 ms, total: 6.69 s
Wall time: 6.6 s


#### Post-processing

Separate the results into input tuples and output dictionaries.

In [8]:
grid_inputs, grid_data = zip(*grid_results)
random_inputs, random_data = zip(*random_results)

np.array(grid_data).shape

(20, 81)

Extract the mean absolute error (MAE) from each of the output dictionaries.

In [9]:
grid_mae = np.array([[g["mae"] for g in gd] for gd in grid_data])
random_mae = np.array([[r["mae"] for r in rd] for rd in random_data])

grid_mae.shape

(20, 81)

Compute statistics (mean and standard deviation) across the `num_repeats` campaigns.

In [10]:
grid_running_mae = np.minimum.accumulate(grid_mae, axis=1)
grid_avg_mae = np.mean(grid_running_mae, axis=0)
grid_std_mae = np.std(grid_running_mae, axis=0)

random_running_mae = np.minimum.accumulate(random_mae, axis=1)
random_avg_mae = np.mean(random_running_mae, axis=0)
random_std_mae = np.std(random_running_mae, axis=0)

grid_avg_mae.shape

(81,)

In [11]:
grid_best_mae = np.min(grid_mae, axis=1)
random_best_mae = np.min(random_mae, axis=1)

best_df = pd.DataFrame(dict(grid=grid_best_mae, random=random_best_mae))
best_df

,grid,random
0,0.001902,0.000500
1,0.000846,0.000119
2,0.000538,0.000155
3,0.002692,0.001231
4,0.001726,0.001312
5,0.000619,0.001446
6,0.000422,0.000342
7,0.001151,0.000366
8,0.000959,0.000281
9,0.001460,0.000606


In [12]:
pd.options.display.float_format = '{:.6f}'.format
best_df.agg([np.mean, np.std])

,grid,random
mean,0.001167,0.000630
std,0.000675,0.000530


In [38]:
def prep_df_for_lines(methods, avg_maes, std_maes):
    method_dfs = []
    for method, avg_mae, std_mae in zip(methods, avg_maes, std_maes):
        method_df = (
            pd.DataFrame(
                {"avg_mae": np.minimum.accumulate(avg_mae), "std_mae": std_mae}
            )
            .reset_index()
            .rename(columns={"index": "iteration"})
        )
        method_df["method"] = method
        method_dfs.append(method_df)
    return pd.concat(method_dfs).reset_index(drop=True)


df = prep_df_for_lines(
    ["grid", "random"],
    [grid_avg_mae, random_avg_mae],
    [grid_std_mae, random_std_mae],
)
df


,iteration,avg_mae,std_mae,method
0,0,0.003858,0.002651,grid
1,1,0.003858,0.002651,grid
2,2,0.003858,0.002651,grid
3,3,0.003858,0.002651,grid
4,4,0.002866,0.002234,grid
...,...,...,...,...
157,76,0.000630,0.000517,random
158,77,0.000630,0.000517,random
159,78,0.000630,0.000517,random
160,79,0.000630,0.000517,random


Here, we plot the average and standard deviation "best objective vs. iteration" for grid
and random search.

In [39]:
from self_driving_lab_demo.utils.plotting import line

line(
    data_frame=df,
    x="iteration",
    y="avg_mae",
    error_y="std_mae",
    error_y_mode="band",
    color="method",
)


We can also have a look at the individual optimization traces.

In [40]:
import plotly
import plotly.express as px
import plotly.graph_objects as go

def legendgroup_lines(running_names, running_maes):
    fig = go.Figure()
    rgb_colors = np.array(
        [plotly.colors.hex_to_rgb(hex) for hex in px.colors.qualitative.Plotly]
    )
    transparency = 0.5 * np.ones((rgb_colors.shape[0], 1))
    rgba_colors = np.concatenate((rgb_colors, transparency), axis=1)
    rgba_colors = [f"rgba{tuple(c)}" for c in rgba_colors]

    for i, (name, running_mae) in enumerate(zip(running_names, running_maes)
    ):
        for j, grm in enumerate(running_mae):
            fig.add_trace(
                go.Scatter(
                    y=grm,
                    mode="lines",
                    line=dict(color=rgba_colors[i]),
                    legendgroup=name,
                    name=name,
                    showlegend=True if j == 0 else False,
                )
            )
    return fig
# fig.update_layout(showlegend=False)
fig = legendgroup_lines(["grid", "random"], [grid_running_mae, random_running_mae])
fig.show()


## Bayesian Optimization

Since Meta's Adaptive Experimentation (Ax) platform takes much longer on hardware
without MKL optimization (e.g. RPi), we'll use a different package for Bayesian
optimization (`scikit-optimize`) in hopes that it will run quicker for this demo.

We'll use the [`forest_minimize`](https://scikit-optimize.github.io/stable/modules/generated/skopt.forest_minimize.html#skopt.forest_minimize)* function and
[`use_named_args`](https://scikit-optimize.github.io/stable/modules/generated/skopt.utils.use_named_args.html#skopt.utils.use_named_args)
decorator.

<p>
<sup>
*typically faster evaluation but less efficient searching than <a href=https://scikit-optimize.github.io/stable/modules/generated/skopt.gp_minimize.html>gp_minimize</a>
</sup>
</p>

In [19]:
from skopt.space import Real, Integer
from skopt import gp_minimize, forest_minimize
from skopt.utils import use_named_args

Similar to the previous notebook, we define the search space (what
`scikit-optimize` calls `dimensions`). This is necessary for using named parameters,
which is useful for when the search space might change later on.

In [20]:
bounds = sdls[0].bounds
dimensions = [
    Integer(name=name, low=bnd[0], high=bnd[1])
    if isinstance(bnd[0], int)
    else Real(name=name, low=bnd[0], high=bnd[1])
    for name, bnd in bounds.items()
]
dimensions


[Real(low=0.0, high=0.35, prior='uniform', transform='identity'),
 Integer(low=0, high=255, prior='uniform', transform='identity'),
 Integer(low=0, high=255, prior='uniform', transform='identity'),
 Integer(low=0, high=255, prior='uniform', transform='identity')]

Again, similar to the previous notebook, we define our objective function `evaluate`.
This time, we use the `use_named_args` decorator to allow for named parameters. Let's
run the optimization.

In [22]:
%%time
from tqdm import tqdm
bo_results = []
for sdl in tqdm(sdls):

    @use_named_args(dimensions=dimensions)
    def evaluate(brightness, R, G, B):
        return sdl.evaluate(brightness, R, G, B)["mae"]

    bo_results.append(
        forest_minimize(evaluate, dimensions, n_calls=num_iter, verbose=False)
    )

100%|██████████| 20/20 [18:26<00:00, 55.35s/it]

CPU times: user 18min 21s, sys: 2.6 s, total: 18min 24s
Wall time: 18min 27s


Then, we'll post-process the data to get a best-objective-vs-iteration plot.

In [33]:
bayesian_mae = np.array([bor.func_vals for bor in bo_results])
bayesian_mae.shape

(20, 81)

In [35]:
bayesian_running_mae = np.minimum.accumulate(bayesian_mae, axis=1)
bayesian_avg_mae = np.mean(bayesian_running_mae, axis=0)
bayesian_std_mae = np.std(bayesian_running_mae, axis=0)
bayesian_avg_mae.shape

(81,)

In [41]:
df = prep_df_for_lines(
    ["grid", "random", "bayesian"],
    [grid_avg_mae, random_avg_mae, bayesian_avg_mae],
    [grid_std_mae, random_std_mae, bayesian_std_mae],
)
line(
    data_frame=df,
    x="iteration",
    y="avg_mae",
    error_y="std_mae",
    error_y_mode="band",
    color="method",
)


### Example Output

In this case, Bayesian is about on par with random (slightly less efficient) and grid is
somewhat less efficient. Why? I'm not sure. But the problem is a relatively simple linear
combination of colors and noiseless. If noise were to be present (as is the case with
any experiment), then we would need to validate our results afterwards by running
multiple repeats at the best predicted locations and then compare what did the best. For
examples of this type of validation, see
https://dx.doi.org/10.1016/j.commatsci.2022.111505 and http://arxiv.org/abs/2203.12597.

![sensor-simulator](sensor-simulator.png)

We can also take a look at the individual traces as before. Interestingly, sometimes it
takes about 60 iterations before Bayesian optimization drops below `0.003`. Perhaps
changing the objective function to something more meaningful (e.g. Wasserstein distance
between discrete distributions) could help with the search efficiency.

In [43]:
fig = legendgroup_lines(
    ["random", "bayesian"],
    [random_running_mae, bayesian_running_mae],
)
fig.show()


### Example Output

![simulator-random-vs-bayesian.png](simulator-random-vs-bayesian.png)

TODO: validation and comparison of the best parameters

## Up Next

[`2.3-bayesian-optimization.ipynb`](2.3-bayesian-optimization.ipynb)
> 🔁 Back to the algorithm comparison experiments! Lo and behold, Bayesian
> optimization is the most efficient. 😌

## Code Graveyard

~~So, for this simulation, grid search seems to be outperforming random search!*~~


~~*Typically,
random search is better, so I dug back into the `get_random_inputs` function from
`SelfDrivingLabDemo`, and it turns out there was a bug
`R, G, B = RGB.astype(int)` -> `R, G, B = np.round(RGB).astype(int)`. However, the results were the same after re-running the experiment.~~

In [ ]:
# import plotly.graph_objs as go

# x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# y = [1, 2, 7, 4, 5, 6, 7, 8, 9, 10]
# y_upper = [2, 3, 8, 5, 6, 7, 8, 9, 10, 11]
# y_lower = [0, 1, 5, 3, 4, 5, 6, 7, 8, 9]

# avg_mae = grid_avg_mae
# std_mae = grid_std_mae

# fig = go.Figure([
#     go.Scatter(
#         x=x,
#         y=avg_mae,
#         line=dict(color='rgb(0,100,80)'),
#         mode='lines'
#     ),
#     go.Scatter(
#         x=x+x[::-1], # x, then x reversed
#         y=y_upper+y_lower[::-1], # upper, then lower reversed
#         fill='toself',
#         fillcolor='rgba(0,100,80,0.2)',
#         line=dict(color='rgba(255,255,255,0)'),
#         hoverinfo="skip",
#         showlegend=False
#     )
# ])
# fig.show()



# import plotly.graph_objs as go
# import pandas as pd

# avg_mae = grid_avg_mae
# std_mae = grid_std_mae

# df = pd.DataFrame({"avg_mae": np.minimum.accumulate(avg_mae), "std_mae": std_mae})

# x_name = "iteration"
# y_name = "avg_mae"
# y_std_name = "std_mae"

# fig = go.Figure([
#     go.Scatter(
#         name='Measurement',
#         y=df[y_name],
#         mode='lines',
#         line=dict(color='rgb(31, 119, 180)'),
#     ),
#     go.Scatter(
#         name='Upper Bound',
#         y=df[y_name]+df[y_std_name],
#         mode='lines',
#         marker=dict(color="#444"),
#         line=dict(width=0),
#         showlegend=False
#     ),
#     go.Scatter(
#         name='Lower Bound',
#         y=df[y_name]-df[y_std_name],
#         # marker=dict(color="#444"),
#         line=dict(width=0),
#         mode='lines',
#         fillcolor='rgba(68, 68, 68, 0.3)',
#         fill='tonexty',
#         showlegend=False
#     )
# ])
# fig.update_layout(
#     xaxis_title="Iteration",
#     yaxis_title='Mean Absolute Error (unitless)',
#     hovermode="x"
# )
# fig.show()


# [
#     fig.add_trace(
#         go.Scatter(
#             y=rrm,
#             mode="lines",
#             line=dict(color=rgba_colors[1]),
#             legendgroup="random",
#             name="random",
#             showlegend=True if i == 0 else False,
#         )
#     )
#     for i, rrm in enumerate(random_running_mae)
# ]

    # random_df = pd.DataFrame(
    #     {"avg_mae": np.minimum.accumulate(random_avg_mae), "std_mae": random_std_mae}
    # ).reset_index().rename(columns={"index": "iteration"})
    # random_df["method"] = "random"

In [ ]:
# dimensions = [tuple(bv) for bv in bounds.values()]